In [1]:
!wget https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q3.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2019Q4.csv.zip
!wget https://resources.lendingclub.com/LoanStats_2020Q1.csv.zip

--2022-04-22 16:10:52--  https://resources.lendingclub.com/LoanStats_2019Q1.csv.zip
Resolving resources.lendingclub.com... 44.240.114.228, 44.233.1.70, 44.224.144.57
Connecting to resources.lendingclub.com|44.240.114.228|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/zip]
Saving to: 'LoanStats_2019Q1.csv.zip'

LoanStats_2019Q1.cs     [    <=>             ]  19.68M  2.78MB/s    in 8.4s    

2022-04-22 16:11:03 (2.35 MB/s) - 'LoanStats_2019Q1.csv.zip' saved [20632440]

--2022-04-22 16:11:03--  https://resources.lendingclub.com/LoanStats_2019Q2.csv.zip
Resolving resources.lendingclub.com... 44.233.1.70, 44.224.144.57, 44.240.114.228
Connecting to resources.lendingclub.com|44.233.1.70|:443... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/zip]
Saving to: 'LoanStats_2019Q2.csv.zip'

LoanStats_2019Q2.cs     [              <=>   ]  22.44M  1.77MB/s    in 16s     

2022-04-22 16:11:22 (1.40 MB/s) - 'Loa

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership", "annual_inc", 
    "verification_status", "pymnt_plan", "dti", "delinq_2yrs", 
    "inq_last_6mths", "open_acc", "pub_rec", "revol_bal", "total_acc", 
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt", 
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", 
    "total_rec_late_fee", "recoveries", "collection_recovery_fee", 
    "last_pymnt_amnt", "collections_12_mths_ex_med", "policy_code", 
    "application_type", "acc_now_delinq", "tot_coll_amt", "tot_cur_bal", 
    "open_acc_6m", "open_act_il", "open_il_12m", "open_il_24m", 
    "mths_since_rcnt_il", "total_bal_il", "il_util", "open_rv_12m", 
    "open_rv_24m", "max_bal_bc", "all_util", "total_rev_hi_lim", "inq_fi", 
    "total_cu_tl", "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", 
    "bc_open_to_buy", "bc_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "mo_sin_old_il_acct", "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", 
    "mo_sin_rcnt_tl", "mort_acc", "mths_since_recent_bc", 
    "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl", 
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0", "num_sats", 
    "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m", 
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", 
    "pub_rec_bankruptcies", "tax_liens", "tot_hi_cred_lim", 
    "total_bal_ex_mort", "total_bc_limit", "total_il_high_credit_limit", 
    "hardship_flag", "debt_settlement_flag",
    "loan_status"
]

target = "loan_status"

In [4]:
# Load the data
df1 = pd.read_csv(Path("/Users/saraharayratti/Desktop/LendingClub/Resources/Generator/LoanStats_2019Q1.csv.zip"), skiprows=1)[:-2]
df2 = pd.read_csv(Path("/Users/saraharayratti/Desktop/LendingClub/Resources/Generator/LoanStats_2019Q2.csv.zip"), skiprows=1)[:-2]
df3 = pd.read_csv(Path("/Users/saraharayratti/Desktop/LendingClub/Resources/Generator/LoanStats_2019Q3.csv.zip"), skiprows=1)[:-2]
df4 = pd.read_csv(Path("/Users/saraharayratti/Desktop/LendingClub/Resources/Generator/LoanStats_2019Q4.csv.zip"), skiprows=1)[:-2]

df = pd.concat([df1, df2, df3, df4]).loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)


low_risk_rows = df[df[target] == 'low_risk']
high_risk_rows = df[df[target] == 'high_risk']

#df = pd.concat([low_risk_rows, high_risk_rows.sample(n=len(low_risk_rows), replace=True)])
df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=42), high_risk_rows])
df = df.reset_index(drop=True)
df = df.rename({target:'target'}, axis="columns")
df

/Users/saraharayratti/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,123,124,125,128,129,130,133) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/saraharayratti/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,123,124,125,128,129,130,133,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/saraharayratti/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,25000.0,0.2000,662.35,RENT,45000.0,Verified,n,22.40,2.0,0.0,...,25.0,0.0,0.0,41532.0,12941.0,18000.0,20632.0,N,N,low_risk
1,35000.0,0.1862,900.62,MORTGAGE,130000.0,Not Verified,n,17.71,0.0,0.0,...,88.9,0.0,0.0,401978.0,69537.0,65200.0,19056.0,N,N,low_risk
2,6400.0,0.0881,202.96,RENT,70000.0,Not Verified,n,3.27,0.0,0.0,...,0.0,1.0,0.0,23400.0,3571.0,15800.0,4000.0,N,N,low_risk
3,7000.0,0.2000,260.15,RENT,65000.0,Verified,n,15.66,0.0,0.0,...,50.0,0.0,0.0,64853.0,53589.0,5300.0,57453.0,N,N,low_risk
4,8500.0,0.1430,291.75,OWN,41000.0,Not Verified,n,24.18,0.0,0.0,...,25.0,1.0,0.0,37469.0,19623.0,16000.0,18169.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13725,14400.0,0.1430,494.26,MORTGAGE,75000.0,Not Verified,n,22.50,0.0,0.0,...,50.0,0.0,0.0,305090.0,36244.0,15100.0,53974.0,N,N,high_risk
13726,15000.0,0.2305,423.29,RENT,83000.0,Source Verified,n,17.58,0.0,1.0,...,80.0,0.0,0.0,146249.0,133217.0,14900.0,127049.0,N,N,high_risk
13727,39000.0,0.1033,1264.48,OWN,70000.0,Not Verified,n,18.02,0.0,1.0,...,20.0,0.0,0.0,312266.0,45852.0,28400.0,42845.0,N,N,high_risk
13728,20925.0,0.0819,657.55,MORTGAGE,50400.0,Not Verified,n,20.60,0.0,0.0,...,0.0,0.0,0.0,177602.0,24039.0,25900.0,34093.0,N,N,high_risk


In [5]:
df.to_csv("../2019loans.csv", index=False)

In [6]:
# Load the data
validate_df = pd.read_csv(Path("/Users/saraharayratti/Desktop/LendingClub/Resources/Generator/LoanStats_2020Q1.csv.zip"), skiprows=1)[:-2]
validate_df = validate_df.loc[:, columns].copy()

# Drop the null columns where all values are null
validate_df = validate_df.dropna(axis="columns", how="all")

# Drop the null rows
validate_df = validate_df.dropna()

# Remove the `Issued` loan status
issued_mask = validate_df[target] != "Issued"
validate_df = validate_df.loc[issued_mask]

# convert interest rate to numerical
validate_df["int_rate"] = validate_df["int_rate"].str.replace("%", "")
validate_df["int_rate"] = validate_df["int_rate"].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = dict.fromkeys(["Current", "Fully Paid"], "low_risk") 
validate_df = validate_df.replace(x)
x = dict.fromkeys(["Late (31-120 days)", "Late (16-30 days)", "Default", "In Grace Period", "Charged Off"], "high_risk")    
validate_df = validate_df.replace(x)

low_risk_rows = validate_df[validate_df[target] == "low_risk"]
high_risk_rows = validate_df[validate_df[target] == "high_risk"]

validate_df = pd.concat([low_risk_rows.sample(n=len(high_risk_rows), random_state=37), high_risk_rows])
validate_df = validate_df.reset_index(drop=True)
validate_df = validate_df.rename({target:"target"}, axis="columns")
validate_df

/Users/saraharayratti/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,138,139,140) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target
0,8000.0,0.0819,251.40,MORTGAGE,53000.0,Not Verified,n,30.87,0.0,1.0,...,0.0,0.0,0.0,214538.0,52345.0,500.0,67066.0,N,N,low_risk
1,30000.0,0.1102,652.58,MORTGAGE,120000.0,Not Verified,n,21.53,0.0,0.0,...,0.0,0.0,0.0,554901.0,101234.0,42500.0,119678.0,N,N,low_risk
2,16000.0,0.0819,325.88,MORTGAGE,95000.0,Verified,n,23.37,0.0,1.0,...,42.9,1.0,0.0,292025.0,76609.0,29900.0,62902.0,N,N,low_risk
3,3000.0,0.1524,104.35,MORTGAGE,50000.0,Not Verified,n,22.30,0.0,0.0,...,75.0,0.0,0.0,472470.0,219678.0,48100.0,171741.0,N,N,low_risk
4,10000.0,0.2305,282.20,OWN,34000.0,Not Verified,n,5.58,0.0,2.0,...,0.0,0.0,0.0,14729.0,5309.0,12800.0,1929.0,N,N,low_risk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13519,30000.0,0.2055,1123.34,RENT,180000.0,Source Verified,n,12.06,0.0,0.0,...,33.3,0.0,0.0,218686.0,209389.0,5600.0,209986.0,N,N,high_risk
13520,17000.0,0.1524,591.32,RENT,240000.0,Source Verified,n,15.88,0.0,2.0,...,16.7,0.0,0.0,151330.0,113872.0,9800.0,140230.0,N,N,high_risk
13521,25000.0,0.2565,1002.62,RENT,60000.0,Source Verified,n,22.44,0.0,0.0,...,62.5,0.0,0.0,57163.0,42830.0,16000.0,41163.0,N,N,high_risk
13522,25000.0,0.1862,911.61,MORTGAGE,160000.0,Not Verified,n,10.12,0.0,3.0,...,66.7,1.0,0.0,55191.0,27274.0,5300.0,48191.0,N,N,high_risk


In [7]:
validate_df.to_csv("../2020Q1loans.csv", index=False)